In [11]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [12]:
final_df = pd.read_csv("merged/final_center.csv")

# Review the DataFrame
final_df.head()

,Unnamed: 0,CTR_CD,CTR_TY,RELEASE_DATE,ORG,RCC_A10_C,RCC_A17_C,RCC_A2_C,RCC_A34_C,RCC_A49_C,...,RCC_TST1B_C,RCC_TST1B_R,RCC_TST2_C,RCC_TST2_R,RCC_TINA_C,RCC_TINA_R,RCC_N_C,D_OR_L,YEAR,MONTH
0,1,ALUA,TX1,2012/07/13,Heart,15.6,15.6,0.0,3.1,9.4,...,43.8,36.1,3.1,4.4,0.0,0.0,32,D,2012,July
1,2,ARBH,TX1,2012/07/13,Heart,0.0,0.0,0.0,12.5,0.0,...,37.5,36.1,0.0,4.4,0.0,0.0,8,D,2012,July
2,3,ARCH,TX1,2012/07/13,Heart,32.3,19.4,48.4,0.0,0.0,...,0.0,36.1,6.5,4.4,0.0,0.0,31,D,2012,July
3,4,AZCH,TX1,2012/07/13,Heart,0.0,50.0,50.0,0.0,0.0,...,0.0,24.3,0.0,13.5,0.0,0.0,2,D,2012,July
4,5,AZMC,TX1,2012/07/13,Heart,0.0,0.0,0.0,0.0,14.3,...,42.9,24.3,0.0,13.5,0.0,0.0,14,D,2012,July


In [13]:
final_df_dummies = pd.get_dummies(final_df)
final_df_dummies.head()

,Unnamed: 0,RCC_A10_C,RCC_A17_C,RCC_A2_C,RCC_A34_C,RCC_A49_C,RCC_A65P_C,RCC_BAB_C,RCC_BA_C,RCC_BB_C,...,ORG_ Heart,ORG_HR,D_OR_L_D,MONTH_August,MONTH_December,MONTH_January,MONTH_July,MONTH_June,MONTH_May,MONTH_November
0,1,15.6,15.6,0.0,3.1,9.4,6.3,3.1,37.5,15.6,...,1,0,1,0,0,0,1,0,0,0
1,2,0.0,0.0,0.0,12.5,0.0,0.0,0.0,37.5,12.5,...,1,0,1,0,0,0,1,0,0,0
2,3,32.3,19.4,48.4,0.0,0.0,0.0,9.7,32.3,12.9,...,1,0,1,0,0,0,1,0,0,0
3,4,0.0,50.0,50.0,0.0,0.0,0.0,50.0,50.0,0.0,...,1,0,1,0,0,0,1,0,0,0
4,5,0.0,0.0,0.0,0.0,14.3,14.3,7.1,35.7,7.1,...,1,0,1,0,0,0,1,0,0,0


In [14]:
inertia = []
# Create a a list to store the values of k
k = list(range(1, 11))

# Create a for loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the home_sales_df DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the K-means model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(final_df_dummies)
    inertia.append(k_model.inertia_)

# Create a dictionary that holds the list values for k and inertia
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame using the elbow_data dictionary
df_elbow = pd.DataFrame(elbow_data)

# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)



:Curve   [k]   (inertia)

In [17]:
# Initialize the K-Means model with n_clusters=3
model = KMeans(n_clusters=4)

# Fit the model for the rate_df_scaled DataFrame
model.fit(final_df_dummies)

# Save the predicted model clusters to a new DataFrame.
clusters = model.predict(final_df_dummies)


print(clusters)

[1 1 3 ... 0 0 0]


In [18]:
final_df["clusters"] = clusters
final_df_grouped = final_df.groupby(['clusters']).mean()
final_df_grouped.hvplot.bar(
    x="clusters", 
    y="RCC_A65P_C", 
    title="avg 65 plus patient in each group", 
)

/var/folders/n3/0fjc3g9d04gb6vgxphygsqj80000gn/T/ipykernel_90981/3543097699.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_df_grouped = final_df.groupby(['clusters']).mean()


:Bars   [clusters]   (RCC_A65P_C)